# HIV Clinical Decision Support Tools

**Research Package: HIV Treatment Optimization**

This notebook provides interactive tools for:
1. **H6: TDR Screening** - Transmitted Drug Resistance detection for treatment-naive patients
2. **H7: LA Injectable Selection** - Long-acting injectable eligibility assessment

---

## Overview

### Clinical Context

| Tool | Purpose | Clinical Impact |
|------|---------|----------------|
| **TDR Screening** | Detect pre-treatment resistance | Guide first-line regimen selection |
| **LA Selection** | Assess CAB-LA/RPV-LA eligibility | Identify candidates for monthly injections |

### Key Mutations

- **NRTI**: M184V/I (3TC/FTC), K65R (TDF), T215Y/F (AZT)
- **NNRTI**: K103N (EFV/NVP), Y181C (NVP), E138K (RPV)
- **INSTI**: Q148H/R (RAL/EVG/DTG), N155H (RAL/EVG)

In [ ]:
# Setup and imports
import sys
from pathlib import Path
import json
import numpy as np
import pandas as pd

# Add package paths
package_dir = Path().absolute().parent
deliverables_dir = package_dir.parent.parent
sys.path.insert(0, str(deliverables_dir))
sys.path.insert(0, str(package_dir / "scripts"))

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("HIV Clinical Tools Notebook Ready")
print(f"Package directory: {package_dir}")

---

## Part 1: TDR Mutation Database

### WHO Surveillance Drug Resistance Mutations (SDRM)

In [ ]:
# Define TDR mutation database
TDR_MUTATIONS = {
    "NRTI": {
        "M184V": {"drugs": ["3TC", "FTC"], "level": "high", "prevalence": 5.2},
        "M184I": {"drugs": ["3TC", "FTC"], "level": "high", "prevalence": 0.8},
        "K65R": {"drugs": ["TDF", "ABC"], "level": "moderate", "prevalence": 2.1},
        "K70R": {"drugs": ["AZT", "TDF"], "level": "moderate", "prevalence": 1.5},
        "L74V": {"drugs": ["ABC", "DDI"], "level": "high", "prevalence": 0.6},
        "T215Y": {"drugs": ["AZT", "D4T"], "level": "high", "prevalence": 2.3},
        "T215F": {"drugs": ["AZT", "D4T"], "level": "high", "prevalence": 1.8},
    },
    "NNRTI": {
        "K103N": {"drugs": ["EFV", "NVP"], "level": "high", "prevalence": 4.8},
        "Y181C": {"drugs": ["NVP", "EFV"], "level": "high", "prevalence": 2.1},
        "G190A": {"drugs": ["NVP", "EFV"], "level": "high", "prevalence": 1.9},
        "K101E": {"drugs": ["NVP", "EFV"], "level": "moderate", "prevalence": 0.8},
        "E138K": {"drugs": ["RPV"], "level": "high", "prevalence": 0.5},
    },
    "INSTI": {
        "N155H": {"drugs": ["RAL", "EVG"], "level": "high", "prevalence": 0.2},
        "Q148H": {"drugs": ["RAL", "EVG", "DTG"], "level": "high", "prevalence": 0.1},
        "Q148R": {"drugs": ["RAL", "EVG", "DTG"], "level": "high", "prevalence": 0.1},
        "Y143R": {"drugs": ["RAL"], "level": "high", "prevalence": 0.1},
    },
}

# Convert to DataFrame for visualization
mutation_rows = []
for drug_class, mutations in TDR_MUTATIONS.items():
    for mut, info in mutations.items():
        mutation_rows.append({
            "Mutation": mut,
            "Drug Class": drug_class,
            "Affected Drugs": ", ".join(info["drugs"]),
            "Resistance Level": info["level"],
            "Prevalence (%)": info["prevalence"]
        })

df_mutations = pd.DataFrame(mutation_rows)
print(f"Total TDR mutations in database: {len(df_mutations)}")
df_mutations.head(10)

In [ ]:
# Visualize TDR prevalence by drug class
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Prevalence by mutation
ax1 = axes[0]
top_mutations = df_mutations.nlargest(10, "Prevalence (%)")
colors = {"NRTI": "#2ecc71", "NNRTI": "#3498db", "INSTI": "#e74c3c"}
bar_colors = [colors[dc] for dc in top_mutations["Drug Class"]]
ax1.barh(top_mutations["Mutation"], top_mutations["Prevalence (%)"], color=bar_colors)
ax1.set_xlabel("Prevalence (%)")
ax1.set_title("Top 10 TDR Mutations by Prevalence")
ax1.invert_yaxis()

# Prevalence by drug class
ax2 = axes[1]
class_prev = df_mutations.groupby("Drug Class")["Prevalence (%)"].sum()
ax2.pie(class_prev.values, labels=class_prev.index, autopct="%1.1f%%",
        colors=[colors[c] for c in class_prev.index])
ax2.set_title("TDR Prevalence by Drug Class")

plt.tight_layout()
plt.show()

---

## Part 2: TDR Screening Tool (H6)

### Analyze Patient Sequence for Drug Resistance

In [ ]:
# Import TDR screening module
try:
    from H6_tdr_screening import (
        TDRResult, detect_mutations, predict_drug_susceptibility,
        recommend_regimen, FIRST_LINE_REGIMENS
    )
    TDR_AVAILABLE = True
    print("TDR Screening module loaded successfully")
except ImportError as e:
    TDR_AVAILABLE = False
    print(f"TDR module not available: {e}")
    print("Using simplified implementation")

In [ ]:
# TDR Analysis Function
from dataclasses import dataclass
from typing import Optional

@dataclass
class TDRResult:
    patient_id: str
    detected_mutations: list
    drug_susceptibility: dict
    tdr_positive: bool
    recommended_regimen: str
    alternative_regimens: list
    confidence: float

# First-line regimens
FIRST_LINE_REGIMENS = [
    {"name": "TDF/3TC/DTG", "drugs": ["TDF", "3TC", "DTG"], "preferred": True},
    {"name": "TDF/FTC/DTG", "drugs": ["TDF", "FTC", "DTG"], "preferred": True},
    {"name": "TAF/FTC/DTG", "drugs": ["TAF", "FTC", "DTG"], "preferred": False},
    {"name": "TDF/3TC/EFV", "drugs": ["TDF", "3TC", "EFV"], "preferred": False},
]

def analyze_tdr(sequence: str, patient_id: str = "DEMO") -> TDRResult:
    """Analyze sequence for transmitted drug resistance."""
    detected = []
    
    # Simulate mutation detection (in production, align to HXB2)
    np.random.seed(hash(sequence[:20]) % 2**32)
    
    for drug_class, mutations in TDR_MUTATIONS.items():
        for mut_name, info in mutations.items():
            if np.random.random() * 100 < info["prevalence"] * 2:  # Increased for demo
                detected.append({
                    "mutation": mut_name,
                    "drug_class": drug_class,
                    "affected_drugs": info["drugs"],
                    "level": info["level"]
                })
    
    # Predict drug susceptibility
    all_drugs = ["TDF", "TAF", "ABC", "3TC", "FTC", "EFV", "NVP", "DTG", "RAL"]
    susceptibility = {drug: {"status": "susceptible", "score": 0} for drug in all_drugs}
    
    for mut in detected:
        for drug in mut["affected_drugs"]:
            if drug in susceptibility:
                susceptibility[drug]["status"] = "resistant" if mut["level"] == "high" else "intermediate"
                susceptibility[drug]["score"] += 30 if mut["level"] == "high" else 15
    
    # Find best regimen
    tdr_positive = len(detected) > 0
    
    valid_regimens = []
    for reg in FIRST_LINE_REGIMENS:
        resistant_drugs = sum(1 for d in reg["drugs"] if susceptibility.get(d, {}).get("status") == "resistant")
        if resistant_drugs == 0:
            valid_regimens.append(reg["name"])
    
    recommended = valid_regimens[0] if valid_regimens else "Refer for expert consultation"
    alternatives = valid_regimens[1:4] if len(valid_regimens) > 1 else []
    
    return TDRResult(
        patient_id=patient_id,
        detected_mutations=detected,
        drug_susceptibility=susceptibility,
        tdr_positive=tdr_positive,
        recommended_regimen=recommended,
        alternative_regimens=alternatives,
        confidence=0.85 if not tdr_positive else 0.75
    )

print("TDR Analysis function ready")

In [ ]:
# Demo: Analyze sample patients
demo_sequences = [
    "PISPIETVPVKLKPGMDGPKVKQWPLTEEKIKALVEICTEMEKEGKISKIGPENPYNTPVFAIKKKDSTKWRKLVDFRELNKRTQDFWEVQLGIPHPAGLKKKKSVTVLDVGDAYFSVPLDKDFRKYTAFTIPSINNETPGIRYQYNVLPQGWKGSPAIFQSSMTKILEPFRKQNPDIVIYQYMDDLYVGSDLEIGQHRTKIEELRQHLLRWGLTTPDKKHQKEPPFLWMGYELHPDKWTVQPIVLPEKDSWTVNDIQKLVGKLNWASQIYPGIKVRQLCKLLRGTKALTEVIPLTEEAELELAENREILKEPVHGVYYDPSKDLIAEIQKQGQGQWTYQIYQEPFKNLKTGKYARMRGAHTNDVKQLTEAVQKITTESIVIWGKTPKFKLPIQKETWETWWTEYWQATWIPEWEFVNTPPLVKLWYQLEKEPIVGAETF",
    "PISPIETVPVKLKPGMDGPKVKQWPLTEEKIKALVEICTEMEKEGKISKIGPENPYNTPVFAIKKKDSTKWRKLVDFRELNKRTQDFWEVQLGIPHPAGLKKKKSVTVLDVGDAYFSVPLDKDFRKYTAFTIPSINNETPGIRYQYNVLPQGWKGSPAIFQSSMTKILEPFRKQNPDIVIYQYMDDLYVGSDLEIGQHRTKIEELRQHLLRWGLTTPDKKHQKEPPFLWMGYELHPDKWTVQPIVLPEKDSWTVNDIQKLVGKLNWASQIYPGIKVRQLCKLLRGTKALTEVIPLTEEAELELAENREILKEPVHGVYYDPSKDLIAEIQKQGQGQWTYQIYQEPFKNLKTGKYARMRGAHTNDVKQLTEAVQKITTESIVIWGKTPKFKLPIQKETWETWWTEYWQATWIPEWEFVNTPPLVKLWYQLEKEPIVGAETF",
    "CCTCAGGTCACTCTTTGGCAACGACCCCTCGTCACAATAAAGATAGGGGGGCAACTAAAGGAAGCTCTATTAGATACAGGAGCAGATGATACAGTATTAGAAGAAATGAGTTTGCCAGGAAGATGGAAACCAAAAATGATAGGGGGAATTGGAGGTTTTATCAAAGTAAGACAGTATGATCAGATACTCATAGAAATCTGTGGACATAAAGCTATAGGTACAGTATTAGTAGGACCTACACCTGTCAACATAATTGGAAGAAATCTGTTGACTCAGATTGGTTGCACTTTAAATTTT"
]

print("=" * 60)
print("TDR SCREENING RESULTS")
print("=" * 60)

results = []
for i, seq in enumerate(demo_sequences, 1):
    result = analyze_tdr(seq, f"Patient_{i:03d}")
    results.append(result)
    
    print(f"\n--- {result.patient_id} ---")
    print(f"TDR Status: {'POSITIVE' if result.tdr_positive else 'NEGATIVE'}")
    print(f"Mutations detected: {len(result.detected_mutations)}")
    for mut in result.detected_mutations:
        print(f"  - {mut['mutation']} ({mut['drug_class']}): affects {', '.join(mut['affected_drugs'])}")
    print(f"Recommended regimen: {result.recommended_regimen}")
    print(f"Alternatives: {', '.join(result.alternative_regimens) if result.alternative_regimens else 'None'}")

In [ ]:
# Visualize drug susceptibility
def plot_susceptibility(result: TDRResult):
    """Plot drug susceptibility heatmap."""
    drugs = list(result.drug_susceptibility.keys())
    scores = [result.drug_susceptibility[d]["score"] for d in drugs]
    statuses = [result.drug_susceptibility[d]["status"] for d in drugs]
    
    colors = {"susceptible": "#2ecc71", "intermediate": "#f39c12", "resistant": "#e74c3c"}
    bar_colors = [colors[s] for s in statuses]
    
    fig, ax = plt.subplots(figsize=(10, 4))
    bars = ax.bar(drugs, scores, color=bar_colors)
    
    ax.set_ylabel("Resistance Score")
    ax.set_title(f"Drug Susceptibility Profile - {result.patient_id}")
    ax.axhline(y=30, color="orange", linestyle="--", alpha=0.5, label="Intermediate threshold")
    ax.axhline(y=60, color="red", linestyle="--", alpha=0.5, label="Resistant threshold")
    ax.legend()
    
    # Add status labels
    for bar, status in zip(bars, statuses):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2,
               status[0].upper(), ha="center", fontsize=9)
    
    plt.tight_layout()
    return fig

# Plot for first patient with mutations
for result in results:
    if result.tdr_positive:
        plot_susceptibility(result)
        plt.show()
        break

---

## Part 3: Long-Acting Injectable Selection (H7)

### Assess Eligibility for CAB-LA/RPV-LA

In [ ]:
# LA Injectable mutations and risk factors
LA_DRUGS = {
    "CAB": {
        "name": "Cabotegravir",
        "class": "INSTI",
        "mutations": {
            "Q148H": 5.0, "Q148R": 3.0, "N155H": 2.5,
            "G140S": 2.0, "E138K": 1.5
        }
    },
    "RPV": {
        "name": "Rilpivirine",
        "class": "NNRTI",
        "mutations": {
            "E138K": 3.0, "E138A": 2.5, "K101E": 2.0,
            "Y181C": 5.0, "Y181I": 4.0, "H221Y": 2.5
        }
    }
}

@dataclass
class PatientData:
    patient_id: str
    age: int
    bmi: float
    viral_load: float
    cd4_count: int
    prior_nnrti: bool
    adherence: str  # excellent, good, moderate, poor
    psychiatric_history: bool

@dataclass
class LASelectionResult:
    patient_id: str
    eligible: bool
    success_probability: float
    cab_risk: float
    rpv_risk: float
    pk_adequacy: float
    risk_factors: list
    recommendation: str

def assess_la_eligibility(patient: PatientData, mutations: list = None) -> LASelectionResult:
    """Assess patient eligibility for LA injectables."""
    risk_factors = []
    
    # Check viral suppression (must be <50)
    if patient.viral_load >= 50:
        risk_factors.append("Not virally suppressed")
    
    # Check BMI impact on PK
    pk_multiplier = 1.0
    if patient.bmi > 30:
        pk_multiplier = 0.85
        risk_factors.append(f"Elevated BMI ({patient.bmi:.1f})")
    if patient.bmi > 35:
        pk_multiplier = 0.70
    
    # Check prior NNRTI (archived resistance risk)
    if patient.prior_nnrti:
        risk_factors.append("Prior NNRTI exposure")
    
    # Check adherence
    adherence_scores = {"excellent": 1.0, "good": 0.9, "moderate": 0.75, "poor": 0.5}
    adherence_score = adherence_scores.get(patient.adherence, 0.75)
    if patient.adherence in ["moderate", "poor"]:
        risk_factors.append(f"Adherence concerns ({patient.adherence})")
    
    # Check psychiatric history
    if patient.psychiatric_history:
        risk_factors.append("Psychiatric history")
    
    # Calculate mutation risk
    cab_risk = 0.0
    rpv_risk = 0.0
    if mutations:
        for mut in mutations:
            if mut["mutation"] in LA_DRUGS["CAB"]["mutations"]:
                cab_risk += LA_DRUGS["CAB"]["mutations"][mut["mutation"]] * 10
            if mut["mutation"] in LA_DRUGS["RPV"]["mutations"]:
                rpv_risk += LA_DRUGS["RPV"]["mutations"][mut["mutation"]] * 10
    
    # Eligibility criteria
    eligible = (
        patient.viral_load < 50 and
        cab_risk < 30 and
        rpv_risk < 30 and
        pk_multiplier > 0.6
    )
    
    # Success probability
    base_success = 0.95
    success_prob = base_success * pk_multiplier * adherence_score
    success_prob *= (1 - cab_risk/100) * (1 - rpv_risk/100)
    if patient.prior_nnrti:
        success_prob *= 0.85
    
    # Recommendation
    if eligible and success_prob > 0.85:
        recommendation = "ELIGIBLE - Good candidate for LA injectables"
    elif eligible and success_prob > 0.70:
        recommendation = "ELIGIBLE WITH MONITORING - Consider enhanced monitoring"
    else:
        recommendation = "NOT RECOMMENDED - Continue oral therapy"
    
    return LASelectionResult(
        patient_id=patient.patient_id,
        eligible=eligible,
        success_probability=success_prob,
        cab_risk=cab_risk,
        rpv_risk=rpv_risk,
        pk_adequacy=pk_multiplier * 100,
        risk_factors=risk_factors,
        recommendation=recommendation
    )

print("LA Selection function ready")

In [ ]:
# Demo: Assess sample patients
demo_patients = [
    PatientData("P001", 35, 24.5, 20, 650, False, "excellent", False),
    PatientData("P002", 42, 32.1, 150, 480, True, "good", False),
    PatientData("P003", 28, 22.0, 30, 720, False, "excellent", True),
    PatientData("P004", 55, 28.5, 80, 520, True, "moderate", False),
    PatientData("P005", 38, 36.5, 25, 600, False, "good", False),
]

print("=" * 60)
print("LA INJECTABLE ELIGIBILITY ASSESSMENT")
print("=" * 60)

la_results = []
for patient in demo_patients:
    result = assess_la_eligibility(patient)
    la_results.append(result)
    
    print(f"\n--- {result.patient_id} ---")
    print(f"Eligible: {'YES' if result.eligible else 'NO'}")
    print(f"Success probability: {result.success_probability*100:.1f}%")
    print(f"PK adequacy: {result.pk_adequacy:.1f}%")
    print(f"Risk factors: {', '.join(result.risk_factors) if result.risk_factors else 'None'}")
    print(f"Recommendation: {result.recommendation}")

In [ ]:
# Visualize LA eligibility results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Success probability
ax1 = axes[0]
patient_ids = [r.patient_id for r in la_results]
probs = [r.success_probability * 100 for r in la_results]
colors = ["#2ecc71" if r.eligible else "#e74c3c" for r in la_results]
bars = ax1.bar(patient_ids, probs, color=colors)
ax1.axhline(y=85, color="green", linestyle="--", alpha=0.7, label="Good candidate (85%)")
ax1.axhline(y=70, color="orange", linestyle="--", alpha=0.7, label="Consider monitoring (70%)")
ax1.set_ylabel("Success Probability (%)")
ax1.set_title("LA Injectable Success Probability")
ax1.set_ylim(0, 100)
ax1.legend()

# Eligibility summary
ax2 = axes[1]
eligible_count = sum(1 for r in la_results if r.eligible)
not_eligible_count = len(la_results) - eligible_count
ax2.pie([eligible_count, not_eligible_count], 
        labels=["Eligible", "Not Eligible"],
        colors=["#2ecc71", "#e74c3c"],
        autopct="%1.0f%%",
        explode=(0.05, 0))
ax2.set_title(f"LA Eligibility (n={len(la_results)})")

plt.tight_layout()
plt.show()

---

## Part 4: Clinical Decision Tree

### Treatment Selection Workflow

In [ ]:
def clinical_decision_workflow(sequence: str, patient: PatientData) -> dict:
    """Complete clinical decision workflow."""
    # Step 1: TDR Screening
    tdr_result = analyze_tdr(sequence, patient.patient_id)
    
    # Step 2: LA Eligibility (if TDR negative and suppressed)
    la_result = None
    if not tdr_result.tdr_positive and patient.viral_load < 50:
        la_result = assess_la_eligibility(patient, tdr_result.detected_mutations)
    
    # Step 3: Final recommendation
    if tdr_result.tdr_positive:
        final_recommendation = f"Start {tdr_result.recommended_regimen} (TDR detected)"
        treatment_path = "TDR-guided oral therapy"
    elif la_result and la_result.eligible:
        final_recommendation = "Consider LA injectable (CAB-LA/RPV-LA)"
        treatment_path = "LA injectable candidate"
    else:
        final_recommendation = f"Start {tdr_result.recommended_regimen}"
        treatment_path = "Standard oral therapy"
    
    return {
        "patient_id": patient.patient_id,
        "tdr_screening": {
            "status": "POSITIVE" if tdr_result.tdr_positive else "NEGATIVE",
            "mutations": len(tdr_result.detected_mutations),
            "recommended_oral": tdr_result.recommended_regimen
        },
        "la_eligibility": {
            "assessed": la_result is not None,
            "eligible": la_result.eligible if la_result else None,
            "success_prob": f"{la_result.success_probability*100:.1f}%" if la_result else None
        },
        "treatment_path": treatment_path,
        "final_recommendation": final_recommendation
    }

# Run workflow for demo patients
print("=" * 60)
print("CLINICAL DECISION WORKFLOW")
print("=" * 60)

for i, patient in enumerate(demo_patients[:3]):
    decision = clinical_decision_workflow(demo_sequences[i], patient)
    
    print(f"\n{decision['patient_id']}:")
    print(f"  TDR: {decision['tdr_screening']['status']} ({decision['tdr_screening']['mutations']} mutations)")
    if decision['la_eligibility']['assessed']:
        print(f"  LA Eligible: {'Yes' if decision['la_eligibility']['eligible'] else 'No'} ({decision['la_eligibility']['success_prob']})")
    print(f"  Path: {decision['treatment_path']}")
    print(f"  RECOMMENDATION: {decision['final_recommendation']}")

---

## Part 5: Export Results

### Generate Clinical Reports

In [ ]:
# Export results to JSON
output_dir = package_dir / "results"
output_dir.mkdir(exist_ok=True)

# TDR results
tdr_export = []
for result in results:
    tdr_export.append({
        "patient_id": result.patient_id,
        "tdr_positive": result.tdr_positive,
        "mutations": result.detected_mutations,
        "recommended_regimen": result.recommended_regimen,
        "alternatives": result.alternative_regimens,
        "confidence": result.confidence
    })

with open(output_dir / "tdr_screening_results.json", "w") as f:
    json.dump(tdr_export, f, indent=2)
print(f"Exported TDR results to {output_dir / 'tdr_screening_results.json'}")

# LA results
la_export = []
for result in la_results:
    la_export.append({
        "patient_id": result.patient_id,
        "eligible": result.eligible,
        "success_probability": result.success_probability,
        "risk_factors": result.risk_factors,
        "recommendation": result.recommendation
    })

with open(output_dir / "la_selection_results.json", "w") as f:
    json.dump(la_export, f, indent=2)
print(f"Exported LA results to {output_dir / 'la_selection_results.json'}")

---

## Summary

This notebook demonstrated:

1. **TDR Screening (H6)**
   - Detection of transmitted drug resistance mutations
   - Drug susceptibility prediction
   - First-line regimen recommendation

2. **LA Injectable Selection (H7)**
   - CAB-LA/RPV-LA eligibility assessment
   - Risk factor identification
   - Success probability calculation

3. **Clinical Workflow**
   - Integrated decision tree
   - Treatment path selection
   - Report generation

### Next Steps

- Integrate Stanford HIVdb API for validated resistance interpretation
- Add real sequence alignment to HXB2 reference
- Implement PDF report generation
- Connect to clinical EHR systems